In [1]:
import torch
from torch import nn
from tc_composer.func.activation import Activation, Softmax
from tc_composer.func.affine_transform import AffineTransform
from tc_composer.func.function_with_params import Composition

[INFO] tc_composer.settings - Setting default tensor type: torch.cuda.FloatTensor
[INFO] tc_composer.settings - Setting epsilon: 1e-16
[INFO] tc_composer.settings - Input tensor shape checking: False
[INFO] tc_composer.settings - Saving compiled options in: /home/ubuntu/tc_composer/options


In [2]:
batch_size = 2
in_n = 3
hidden = 5
out_n = 7

inp = torch.randn(batch_size, in_n)

## Compose

In [3]:
affine0 = AffineTransform(in_n=in_n, out_n=hidden)
softmax = Softmax()
affine1 = AffineTransform(in_n=hidden, out_n=out_n)
relu = Activation('relu')

comp = Composition(affine0, softmax, affine1, relu)
print(comp.tc_def)

def Composition(
    float(I,3) input,
    float(5,3) weight,
    float(5) bias,
    float(7,5) weight1,
    float(7) bias1
) -> (max_val, translated, l1norm, output, output1, output2, output3)
{
    output(b, n) +=! input(b, i) * weight(n, i)
    output(b, n) = output(b, n) + bias(n)
    
    max_val(n) max=! output(n, d)
    translated(n, d) = exp(output(n, d) - max_val(n))
    l1norm(n) +=! translated(n, d)
    output1(n, d) = translated(n, d) / l1norm(n)
    
    output2(b, n) +=! output1(b, i) * weight1(n, i)
    output2(b, n) = output2(b, n) + bias1(n)
    
    output3(b, i) = fmax(output2(b, i), 0)
    
}


In [4]:
comp.recompile(inp)

[INFO] Composition - Compiling for input shape - [(2, 3)].
[WARNING] Composition - No option loaded from file for input shape - [(2, 3)].
[WARNING] Composition - Initializing naive options.


## Pytorch

In [5]:
torch_f = nn.Sequential(
    nn.Linear(in_features=in_n, out_features=hidden),
    nn.Softmax(dim=-1),
    nn.Linear(in_features=hidden, out_features=out_n),
    nn.ReLU()
)

## Correctness

In [6]:
for p, (name, tensor) in zip(torch_f.parameters(), comp.named_params):
        p.data = tensor.detach().view_as(p)

In [7]:
import numpy as np

np.testing.assert_allclose(
    comp(inp).cpu().detach().numpy(),
    torch_f(inp).cpu().detach().numpy(),
    rtol=1e-4
)

## More ways to do it

In [8]:
lshift = affine0 << softmax << affine1 << relu
assert lshift.tc_def == comp.tc_def
print(lshift.tc_def)

def Composition(
    float(I,3) input,
    float(5,3) weight,
    float(5) bias,
    float(7,5) weight1,
    float(7) bias1
) -> (max_val, translated, l1norm, output, output1, output2, output3)
{
    output(b, n) +=! input(b, i) * weight(n, i)
    output(b, n) = output(b, n) + bias(n)
    
    max_val(n) max=! output(n, d)
    translated(n, d) = exp(output(n, d) - max_val(n))
    l1norm(n) +=! translated(n, d)
    output1(n, d) = translated(n, d) / l1norm(n)
    
    output2(b, n) +=! output1(b, i) * weight1(n, i)
    output2(b, n) = output2(b, n) + bias1(n)
    
    output3(b, i) = fmax(output2(b, i), 0)
    
}


In [9]:
lshift.recompile(inp)

[INFO] Composition - Compiling for input shape - [(2, 3)].
[WARNING] Composition - No option loaded from file for input shape - [(2, 3)].
[WARNING] Composition - Initializing naive options.


In [10]:
np.testing.assert_allclose(
    lshift(inp).cpu().detach().numpy(),
    torch_f(inp).cpu().detach().numpy(),
    rtol=1e-4
)